# Case Study 1: Two catalysts in a Batch Reactor

The scripts below were used to generate batch reactor data presented in the manuscript. In addition, scripts are included for regression of kinetic parameters and preparation of figures used in the manuscript

In [1]:
#Packages used in this Notebook

using DifferentialEquations
using Printf
using BenchmarkTools
using CairoMakie
using LaTeXStrings
using Polynomials
using DelimitedFiles
using JLD2
using Distributions
using Random
using ComponentArrays
using Parameters

## The Reaction

For this case study, we consider a simple, irreversible reaction performed in a perfectly mixed batch reactor:

$$A \longrightarrow B$$

This specific reaction has elementary kinetics. Assuming thermodynamic ideality (fugacity coefficients for all species are unity) and that the transition state is an adsorbed species, one can develop the following rate expression in turnover frequency units from transition state theory:

$$r^{\prime\prime\prime} = ka_A$$

Where $a_A$ is the thermodynamic activity of species A in the reactor. For a reaction where bulk species are present in a thermodynamically ideal gas phase, we define thermodynamic activity as:

$$a_j = \frac{y_jP}{P^\circ}$$

Because we consider this reaction in a perfectly mixed batch reactor, we can assume that distributions of species are spatially uniform in the batch reactor (thus rates are spatially uniform in the batch reactor).  This allows us to write the following balance on species $j$:

$$\frac{dN_j}{dt} = \nu_j r'''N_S$$

Where $N_j$ is the number of moles of species $j$ in the reactor and $N_S$ is the number of moles of active sites in the reactor.

## System 1: Catalyst 1 with no deactivation

For this case, we will assume that the catalyst is stable such that the number of active sites, $N_S$ is constant at its initial value, $N_{S0}$. That is, the number of active sites is time invariant, giving: 

$$\frac{dN_j}{dt} = \nu_j r'''N_{S0}$$

There are only two quantities that vary in the time domain for this process:  $N_A$ and $N_B$. As such, writing balances on A and B gives a system of ODEs that can be solved to model the molar quantities of A and B as they change with time. 

\begin{align}
    \frac{dN_A}{dt} &= -r'''N_{S0} \\
    \frac{dN_B}{dt} &= r'''N_{S0}
\end{align}

where for Catalyst 1

$$r^{\prime\prime\prime} = k_1a_A$$
    
Note that for this simple example, the ODEs are not coupled, so it is not necessary to solve both simultaneously; however, this is not generally true for complex reaction networks, so we present the solution as though the balances on A and B were coupled ODEs. For this case, we have the following parameters available:

### Available Data

\begin{align}
    T       &= 450.0 \ \mathrm{K} \\
    P       &= 1 \ \mathrm{bar} \\ 
    P^\circ &= 1 \ \mathrm{bar} \\ 
    N_{A0}  &= 10 \ \mathrm{moles \ of \ A}\\
    N_{B0}  &= 0 \ \mathrm{moles \ of \ B} \\
    N_{S0}  &= 1.0 \times 10^{-3} \ \mathrm{moles \ of \ Active \ sites} \\
    A_{1}   &= 1.0 \times 10^{13} \ \mathrm{s^{-1}} \\
    E_{A1}  &= 100 \ \mathrm{kJ \ mol^{-1}}
\end{align}

For these problems, we additionally multiply all simulated data by noise selected randomly from a gaussian distribution with a mean of 1 and a standard deviation of 0.01.

In [2]:
#Specify conditions for the problem
T0    = 450        #K
P     = 1          #bar
P0    = 1          #bar
R     = 0.08314    #L*bar/mol/K
R1    = 8.314      #J/mol/K 
NA0   = 10.0       #moles
NB0   = 0.0        #moles
NT0   = NA0 + NB0  #moles
NS0   = 1.0e-3     #moles
V     = NT0*R*T0/P #L

374.13

### Construct the ODE system that describes batch reactor

The cell below creates a function that captures the behavior of the batch reactor.  Note that Julia allows in-place updating of the ODE function to improve speed. We use this syntax here.  Conventional ODE functions are usually defined in terms of state variables ($u$) and independent variables ($t$):

```
function odefun(u, t)
```

Julia additionally includes an argument for passing parameters ($p$):

```julia
function odefun(u, p, t)
```

Finally, Julia allows in place updating (mutating) of the ODE function, which offers improved speed.  The syntax in this case is:

```julia
function odefun!(du, u, p, t)
```

Where the arguments are `du` is the derivative of state variables; `u` is the value of the state variables; `p` includes any extra parameters; and `t` is the independent variable. The exclamation point in the function definition is not required to use in place updating; however, the syntax convention in Julia is that mutating functions are designated with an exclamation point `!`.

In [3]:
function odebatch1!(du, u, p, t)
    T, P, P0, R, NA0, NB0, NS0, noise, tvals, rseed = p
    
    NA, NB = u       #moles
    NT     = NA + NB #total moles
    NS     = NS0     #moles of sites
        
    yA  = NA/NT      #mole fraction of A
    yB  = NB/NT      #mole fraction of B
            
    pA  = yA*P       #partial pressure of A, bar
    pB  = yB*P       #partial pressure of B, bar
    
    aA  = pA/P0      #thermodynamic activity of A
    aB  = pB/P0      #thermodynamic activity of B
    
    r   = k1(T)*aA   #Turnover Frequency, 1/s

    du[1] = -r*NS    #dNA/dt
    du[2] =  r*NS    #dNB/dt
end

odebatch1! (generic function with 1 method)

### Writing a script to solve the ODE system and process the solution

The cell below includes the code that solves the system of ODEs. Note that here we wrap the ODE solution in a function with various returns, which generally improves performance as the function only need be compiled once and can be solved subsequently for any parameter set by passing that parameter set to the solution function.

In general, ODE solvers in Julia use a logical construction that is similar to ODE solvers in most high level programming languages (e.g., Matlab, Octave, Python). It relies on the use of the `ODEProblem` constructor, which builds the problem (`prob`) that is passed to the solver.  Into this constructor, one provides the name of the function containing the ODEs (`odebatch1!`), the initial values of the state variables (`u0`), the integration span for the independent variable (`tspan`), and any additiona parameters to be passed to the ODEsystem (`par`).

Once the problem is created, a call to `solve(prob)` will solve the ODE system.  As with most ODE solvers, one has options.  Here, we are specifying the use of a Rosenbrock method (`Rosenbrock23`).  We are also including limits on the domain for all state variables through the use of a function that specifies the permissible domain of our state variables.  In this example, since our state variables are all physical quantities (mole numbers), I am specifying that all state variables must be positive at all times.  This is done by creating a domain function (`domainfunc`) that evaluates to `True` if any state variable becomes negative during the course of integration.  If that happens, the solver will return to the last step prior to encountering negative values, take a smaller step, and reevaluate.  This adds some cost to the evaluation as it will generally require taking progressively smaller steps until all state variables fall within the specified domain, but it is beneficial in cases like this one where there is a *physical* restriction on state variables (all molar quantities must be positive), but there is no *mathematical* restriction on their values becoming negative in the course of a numerical integration scheme.  The domainfunction is passed to the ODE solution through the use of the `isoutofdomain` keyword argument.

Once the solution is obtained, Julia permits interpolation of the solution at values inside of the integration span. Below, we use this to generate a simulated set of experimental data by evaluating values of $N_A$ and $N_B$ at 15 data points at specified time values. As is generally the case with most advanced numerical ODE solvers, we don't have control over the exact time steps taken by the solver; however, we are able to specify (by interpolation) the values of the solution at those time steps once the problem is solved.  This is accomplished below by passing the set of target times (`tvals`) to the raw solution structure returned by `solve(prob)`, i.e., `batchsol`.

Once molar quantities for A and B are obtained at desired times by interpolation, these values are used in calculating mole fractions, partial pressures, thermodynamic activities, and reaction rates at each time in `tvals`.

In [4]:
function mainbatch1(par)
    @unpack T, P, P0, R, NA0, NB0, NS0, noise, tvals, rseed = par
    rseed      = Int64.(rseed)
    domainfunc = (u, p, t) -> any(x -> x < 0, u) #ensure all state variables are greater than 0
    u0         = [NA0, NB0]       #initial state of NA and NB, moles
    tspan      = (0, tvals[end])  #integration time span, seconds
    prob       = ODEProblem(odebatch1!, u0, tspan, par) #construct ODE Problem
    batchsol   = solve(prob, Rosenbrock23(), isoutofdomain = domainfunc) #Solve ODE Problem
    tsol       = batchsol.t    #time values used by solver
    NAsol      = batchsol[1,:] #NA at each time value
    NBsol      = batchsol[2,:] #NB at each time value
    yAsol      = NAsol./10 #Mole fraction of A at each time value used by solver
    yBsol      = NBsol./10 #Mole fraction of B at each time value used by solver
    CAsol      = yAsol*P/R/T #CA at each time value used by solver, mol/L
    CBsol      = yBsol*P/R/T #CB at each time value used by solver, mol/L
    
    #display(@benchmark solve($prob, Rosenbrock23(), isoutofdomain = $domainfunc, saveat = $tvals))
    # Create Noisy Data  
    batchout   = batchsol(tvals) #interpolate solution at specific time values
    NA         = batchout[1,:]   #NA at interpolation times, moles
    Random.seed!(rseed[1])
    NA[2:end]  = NA[2:end].*rand(Normal(1.0, noise), length(NA[2:end]))
    NB         = batchout[2,:]   #NB at interpolation times, moles
    Random.seed!(rseed[2])
    NB[2:end]  = NB[2:end].*rand(Normal(1.0, noise), length(NB[2:end]))
    yA         = NA./10 #mole fraction of A at interpolation times
    yB         = NB./10 #mole fraction of B at interpolation times
    pA         = yA*P #partial pressure of A at interpolation times
    pB         = yB*P #partial pressure of B at interpolation times
    aA         = pA/P0 #thermodynamic activity of A at interpolation times
    aB         = pB/P0 #thermodynamic activity of B at interpolation times
    CA         = pA/R/T #concentration of A at interpolation times
    CB         = pB/R/T #concentration of B at interpolation times
    XA         = (NA0 .- NA)/NA0 #Fractional Conversion at interpolation times
    r          = k1(T)*aA #rate of reaction at interpolation times.
    return tvals, NA, NB, CA, CB, aA, aB, XA, r, tsol, NAsol, NBsol
end

mainbatch1 (generic function with 1 method)

### Solving the ODE problem and storing the output

The cell below solves the batch reactor model for the first Catalyst at a given parameter set.

In [5]:
#Solve the problem, store the returns.
A1     = 8.75e13/1000 #Pre-exponential factor, 1/s
EA1    = 90.4*1000 #Barrier, J/mol
tset   = [0, 150, 300, 600, 900, 1200, 1500, 1800, 2100, 2500, 3000, 3500, 4000, 4500, 5300, 6000, 7000, 8000, 9000, 10000] #s
k1(T)  = A1*exp(-EA1/R1/T) #rate constant, 1/s
nlevel = 0.03
# par0   = [T0, P, P0, R, NA0, NB0, NS0, nlevel, tset, [2, 3]]; #initialize parameter set

par0   = ComponentArray(
    T      = T0,
    P      = P,
    P0     = P0,
    R      = R,
    NA0    = NA0,
    NB0    = NB0,
    NS0    = NS0,
    noise  = nlevel,
    tvals  = tset,
    rseed  = [2, 3]
)

t1, NA1, NB1, CA1, CB1, aA1, aB1, XA1, r1, tsol, NAsol, NBsol  = mainbatch1(par0); #run batch simulation, workup data
BATCH1 = ["time (s)" "NA (moles)" "NB (moles)" "CA (mol/L)" "CB (mol/L)" "aA" "aB" "XA" "r (1/s)"]
BATCH1 = vcat(BATCH1, [t1 NA1 NB1 CA1 CB1 aA1 aB1 XA1 r1])

io = open("BATCH1.csv", "w")
writedlm(io, BATCH1, ',')
close(io)

io = open("BR02A.csv", "w")
writedlm(io, BATCH1, ',')
close(io)

In [6]:
# Generate Data for comparing catalysts 1 to 5 (same catalyst, different dispersions/metal loadings)

ρ = [0.25, 0.5, 1, 3, 6]*1e-3 
Nset = 1*ρ
noiseset = [0.0005, 0.00125, 0.0025, 0.02, 0.04]*3
seeds    = [3 4; 5 6; 7 8; 9 10; 11 12]
tVOL = zeros(length(tset), length(ρ))
CVOL = zeros(length(tset), length(ρ))
XVOL = zeros(length(tset), length(ρ))
NVOL = zeros(length(tset), length(ρ))
    
for i = 1:length(ρ)
    par0   = ComponentArray(
        T  = T0,
        P  = P, 
        P0 = P0, 
        R  = R, 
        NA0 = NA0, 
        NB0 = NB0,
        NS0 = Nset[i], 
        noise = noiseset[i], 
        tvals = tset, 
        rseed = seeds[i, :]
    )
    t, NA, NB, CA, CB, aA, aB, XA, r = mainbatch1(par0); #run batch simulation, workup data
    tVOL[:, i] = t
    CVOL[:, i] = CA
    XVOL[:, i] = XA
    NVOL[:, i] = NA
end

BATCH2 = hcat(tVOL, CVOL, XVOL, NVOL)
io = open("BATCH2.csv", "w")
writedlm(io, BATCH2, ',')
close(io)

In [7]:
#Generate Data for all 5 cases at 4 different temperatures
Tset = [400, 425, 450, 475, 500, 525]
noiseset = [0.0005, 0.00125, 0.0025, 0.02, 0.04]
BATCHT = Dict()

for k = 1:length(Tset)
    tVOL = zeros(length(tset), length(ρ))
    CVOL = zeros(length(tset), length(ρ))
    XVOL = zeros(length(tset), length(ρ))
    NVOL = zeros(length(tset), length(ρ))
    for i = 1:length(ρ)
        par0   = ComponentArray(
            T = Tset[k],  
            P = P, 
            P0 = P0, 
            R  = R, 
            NA0 = NA0, 
            NB0 = NB0, 
            NS0 = Nset[i], 
            noise = noiseset[i], 
            tvals = tset, 
            rseed = seeds[i, :]*i
        )
        t, NA, NB, CA, CB, aA, aB, XA, r = mainbatch1(par0); #run batch simulation, workup data
        tVOL[:, i] = t
        CVOL[:, i] = CA
        XVOL[:, i] = XA
        NVOL[:, i] = NA
    end
    BATCHT[string(Tset[k])] = hcat(tVOL, CVOL, XVOL, NVOL)
end

save_object("BATCHT", BATCHT)

## System 2: Catalyst 2 with deactivation

For this case, the catalyst is ***not*** stable. It undergoes some deactivation over the course of the reaction. We describe this deactivation by allowing the number of active sites, $N_S$ to decrease from its starting value, $N_{S0}$ over the course of reaction. This example differs from Case 1 in that the number of active sites is now time dependent, giving: 

$$\frac{dN_j}{dt} = \nu_j r'''N_S(t)$$

$N_A$ and $N_B$ are time dependent, so we write balances in both species as usual.

\begin{align}
    \frac{dN_A}{dt} &= -r'''N_S(t) \\
    \frac{dN_B}{dt} &= r'''N_S(t)
\end{align}

Where the rate of reaction for Catalyst 2 is given in turnover frequency units by:

$$r^{\prime\prime\prime} = k_2a_A$$

In addition, we need a model that describes how the number of active sites is changing as a function of time.  Here, we'll assume first order deactivation; as such, the time derivative of the number of active sites, $N_S$ is proportional to the number of active sites:

$$ \frac{dN_S}{dt} = -k_d N_S$$

Subject to the initial condition that, at time = 0, the number of active sites is $N_{S0}$.  Here, the rate constant has units of inverse seconds, typical of a first order process.

In this example, the balances for $N_A$ and $N_S$ are coupled. In the case of first order deactivation, an analytical solution is tractable, but the more general approach is a numerical solution. We proceed with the numerical solution since it is extensible to a much larger class of reactor modelling problems. For this case, we have the following parameters available:

### Available Data

\begin{align}
    T       &= 450.0 \ \mathrm{K} \\
    P       &= 1 \ \mathrm{bar} \\ 
    P^\circ &= 1 \ \mathrm{bar} \\ 
    N_{A0}  &= 10 \ \mathrm{moles \ of \ A}\\
    N_{B0}  &= 0 \ \mathrm{moles \ of \ B} \\
    N_{S0}  &= 1.0 \times 10^{-3} \ \mathrm{moles \ of \ Active \ sites} \\
    A_{2}   &= 2.5 \times 10^{11} \ \mathrm{s^{-1}} \\
    E_{A2}  &= 88.4 \ \mathrm{kJ \ mol^{-1}} \\
    A_{d}   &= 1.9 \times 10^{10} \ \mathrm{s^{-1}} \\
    E_{Ad}  &= 115 \ \mathrm{kJ \ mol^{-1}} \\
\end{align}

For these problems, we additionally multiply all simulated data by noise selected randomly from a gaussian distribution with a mean of 1 and a standard deviation of 0.01.

In [8]:
function odebatch2!(du, u, p, t)
    @unpack T, P, P0, R, NA0, NB0, NS0, noise, tvals, rseed = p
    
    NA, NB, NS = u # State variables, moles   
    NT         = NA + NB #total moles in bulk = A + B
         
    yA  = NA/NT #mole fraction of A
    yB  = NB/NT #mole fraction of B
    
    pA  = yA*P  #partial pressure of A, bar
    pB  = yB*P  #partial pressure of B, bar
    
    aA  = pA/P0 #thermodynamic activity of A
    aB  = pB/P0 #thermodynamic activity of B
    
    r   = k2(T)*aA            #rate of reaction, TOF units, 1/s
    rd  = kd(T)*(NS - NS0/10) #rate of active site loss, units of moles/time
       
    du[1] = -r*NS #dNA/dt
    du[2] =  r*NS #dNB/dt
    du[3] = -rd   #dNS/dt
end

function mainbatch2(par)
    @unpack T, P, P0, R, NA0, NB0, NS0, noise, tvals, rseed = par
    rseed = Int64.(rseed)
    domainfunc = (u, p, t) -> any(x -> x < 0, u) #keep state variables > 0
    u0 = [NA0, NB0, NS0] #initial state, moles
    tspan = (0.0, tvals[end])  #integration time span
    prob = ODEProblem(odebatch2!, u0, tspan, par) #construct ODE problem
    batchsol = solve(prob, Rosenbrock23(), isoutofdomain = domainfunc) #solve ODE problem
    tsol     = batchsol.t #time steps taken by solver, seconds
    NAsol    = batchsol[1,:] #NA at time steps taken by solver, moles
    NBsol    = batchsol[2,:] #NB at time steps taken by solver, moles
    yAsol    = NAsol./(NAsol + NBsol) #mole fraction of A at time steps in solver
    yBsol    = NBsol./(NAsol + NBsol) #mole fraction of B at time steps in solver
    CAsol    = yAsol*P/R/T            #concentration of A at time steps in solver, mol/L
    CBsol    = yBsol*P/R/T            #concentration of B at time steps in solver, mol/L 

    batchout = batchsol(tvals)        #interpolate ODE solution at specific time points
    NA = batchout[1,:]                #NA at interpolation points, moles
    Random.seed!(rseed[1])
    NA[2:end]  = NA[2:end].*rand(Normal(1.0, noise), length(NA[2:end]))
    NB = batchout[2,:]                #NB at interpolation points, moles
    Random.seed!(rseed[2])
    NB[2:end]  = NB[2:end].*rand(Normal(1.0, noise), length(NB[2:end]))
    NS = batchout[3,:]                #NS at interpolation points
    yA = NA./10                       #mole fraction of A at interpolation points
    yB = NB./10                       #mole fraction of B at interpolation points
    pA = yA*P                         #partial pressure of A at interpolation points, bar
    pB = yB*P                         #partial pressure of B at interpolation points, bar 
    aA = pA/P0                        #thermodynamic activity of A
    aB = pB/P0                        #thermodynamic activity of B
    CA = pA/R/T                       #CA at interpolation points, mol/L
    CB = pB/R/T                       #CB at interpolation points, mol/L
    XA = (NA0 .- NA)/NA0              #Conversion at interpolation points
    r  = k2(T)*aA                     #reaction rate at interpolation points, 1/s
    return tvals, NA, NB, NS, CA, CB, aA, aB, XA, r, tsol, NAsol, NBsol
end

mainbatch2 (generic function with 1 method)

In [9]:
#Solve the problem, store the returns.
A2     = 2.5e14/1000 #pre-exponential for reaction, 1/s
EA2    = 88.4*1000   #Barrier, J/mol
Ad     = 1e13/1000*1.90 #pre-exponential for deactivation, 1/s
EAd    = 115000      #Barrier for deactivation, J/mol
tset   = [0, 50, 100, 200, 300, 400, 500, 700, 900, 1200, 1500, 1800, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000] #s
k2(T)  = A2*exp(-EA2/R1/T) #rate constant for reaction, 1/s
kd(T)  = Ad*exp(-EAd/R1/T) #rate constant for deactivation, 1/s
nlevel = 0.03
par0   = ComponentArray(
    T = T0, 
    P = P, 
    P0 = P0, 
    R  = R, 
    NA0 = NA0, 
    NB0 = NB0, 
    NS0 = NS0, 
    noise = nlevel, 
    tvals = tset, 
    rseed = [14, 15]
    )
t2, NA2, NB2, NS2, CA2, CB2, aA2, aB2, XA2, r2, tsol, NAsol, NBsol = mainbatch2(par0); #solve model for case 2, workup data
BR02B = ["time (s)" "NA (moles)" "NB (moles)" "CA (mol/L)" "CB (mol/L)" "aA" "aB" "XA" "r (1/s)"]
BR02B = vcat(BR02B, [t2 NA2 NB2 CA2 CB2 aA2 aB2 XA2 r2])

io = open("BR02B.csv", "w")
writedlm(io, BR02B, ',')
close(io)

In [10]:
k2(450)

13.688774425900679

In [11]:
tset   = collect(range(0, 10000, 20))
seeds  = [16 17; 18 19; 20 21; 22 23; 24 25; 26 27; 28 29; 30 31; 32 33]
Tset   = collect(400:25:600)
k1out  = zeros(length(Tset))
k2out  = zeros(length(Tset))
XA3max = zeros(length(Tset))
XA4max = zeros(length(Tset))
CairoMakie.activate!(type = "svg")

for i = 1:length(Tset) 
    par0   = ComponentArray(
        T = Tset[i], 
        P = P, 
        P0 = P0, 
        R  = R, 
        NA0 = NA0, 
        NB0 = NB0, 
        NS0 = NS0, 
        noise = nlevel, 
        tvals = tset, 
        rseed = seeds[i,:]
        )
    t3, NA3, NB3, CA3, CB3, aA3, aB3, XA3, r3, tsol3, NAsol3, NBsol3  = mainbatch1(par0);

    par0   = ComponentArray(
        T = Tset[i], 
        P = P, 
        P0 = P0, 
        R  = R, 
        NA0 = NA0, 
        NB0 = NB0, 
        NS0 = NS0, 
        noise = nlevel, 
        tvals = tset, 
        rseed = seeds[i,:]*3
        )
    t4, NA4, NB4, NS4, CA4, CB4, aA4, aB4, XA4, r4, tsol4, NAsol4, NBsol4 = mainbatch2(par0);

    XAsol3    = (NA0 .- NAsol3)/NA0
    index3 = zeros(Bool, size(tsol3, 1))
    index3[1] = true
    for i = 1:length(index3)
        if i % 3 == 0 && XAsol3[i] < 0.9 
            index3[i] = true
        else
            index3[i] = false
        end
    end
    
    X3  = tsol3[index3]
    Y3  = log.(NAsol3[index3]/NA0)
    m3  = transpose(X3)*X3\transpose(X3)*Y3
    k13 = -m3*NT0/NS0*P0/P
    
    XAsol4    = (NA0 .- NAsol4)/NA0
    index4 = zeros(Bool, size(tsol4, 1))
    index4[1] = true
    for i = 1:length(index4)
        if i % 3 == 0 && XAsol4[i] < 0.7 
            index4[i] = true
        else
            index4[i] = false
        end
    end
    
    X4  = tsol4[index4]
    Y4  = 1 ./(NAsol4[index4]) .- 1 ./NA0
    m4  = transpose(X4)*X4\transpose(X4)*Y4
    k24 = m4*NT0^2/NS0*P0^2/P^2
     
    k1out[i] = k13
    k2out[i] = k24
end

f5 = Polynomials.fit(1000 ./Tset, log.(k1out), 1)
f6 = Polynomials.fit(1000 ./Tset, log.(k2out), 1)

A1out  = exp(f5[0])
EA1out = 8.314*f5[1]
A2out  = exp(f6[0])
EA2out = 8.314*f6[1]

[A1out EA1out A2out EA2out]

kOUT = ["T (K)" "k1 (1/s)" "k2 (1/s)"]
kOUT = vcat(kOUT, [Tset k1out k2out])
io = open("BR02C.csv", "w")
writedlm(io, kOUT, ',')
close(io)

In [12]:
#Solve the problem, store the returns.
tset     = [0, 150, 300, 600, 900, 1200, 1500, 1800, 2100, 2500, 3000, 3500, 4000, 4500, 5300, 6000, 7000, 8000, 9000, 10000] #s
t3  = zeros(length(tset), 4)
t3a = zeros(length(tset), 4)
NA3 = zeros(length(tset), 4)
NB3 = zeros(length(tset), 4)
CA3 = zeros(length(tset), 4)
CB3 = zeros(length(tset), 4)
aA3 = zeros(length(tset), 4)
aB3 = zeros(length(tset), 4)
XA3 = zeros(length(tset), 4)
r3  = zeros(length(tset), 4)
rseeds = [20 21; 22 23; 24 25; 26 27]

for i = 1:1:4
    par0   = ComponentArray(
        T = T0, 
        P = P, 
        P0 = P0, 
        R  = R, 
        NA0 = NA0, 
        NB0 = NB0, 
        NS0 = NS0, 
        noise = 0.01, 
        tvals = tset, 
        rseed = rseeds[i, :]
        )
    tvals, NA, NB, CA, CB, aA, aB, XA, rate, tsol, NAsol, NBsol = mainbatch1(par0) #run batch simulation, workup data
    t3[:, i]  = tvals
    t3a[:, i] = tvals .+ tset[end]*(i - 1)
    NA3[:, i] = NA
    NB3[:, i] = NB
    CA3[:, i] = CA
    CB3[:, i] = CB
    aA3[:, i] = aA
    aB3[:, i] = aB
    XA3[:, i] = XA
    r3[:, i]  = rate
end

BR02D = Dict("t" => t3, "ta" => t3a, "NA" => NA3, "CA" => CA3, "aA" => aA3)
save_object("BR02D", BR02D)

In [13]:
function odebatch3!(du, u, p, t)
    @unpack T, P, P0, R, NA0, NB0, NS0, NS0_l, noise, tvals, rseed = p
    
    NA, NB, NS = u # State variables, moles   
    NT         = NA + NB #total moles in bulk = A + B
         
    yA  = NA/NT #mole fraction of A
    yB  = NB/NT #mole fraction of B
    
    pA  = yA*P  #partial pressure of A, bar
    pB  = yB*P  #partial pressure of B, bar
    
    aA  = pA/P0 #thermodynamic activity of A
    aB  = pB/P0 #thermodynamic activity of B
    
    r   = k2(T)*aA            #rate of reaction, TOF units, 1/s
    rd  = kd(T)*(NS - NS0/10) #rate of active site loss, units of moles/time
       
    du[1] = -r*NS #dNA/dt
    du[2] =  r*NS #dNB/dt
    du[3] = -rd   #dNS/dt
end

function mainbatch3(par)
    @unpack T, P, P0, R, NA0, NB0, NS0, NS0_l, noise, tvals, rseed = par
    rseed = Int64.(rseed)
    domainfunc = (u, p, t) -> any(x -> x < 0, u) #keep state variables > 0
    u0 = [NA0, NB0, NS0_l] #initial state, moles
    tspan = (0.0, tvals[end])  #integration time span
    prob = ODEProblem(odebatch3!, u0, tspan, par) #construct ODE problem
    batchsol = solve(prob, Rosenbrock23(), isoutofdomain = domainfunc) #solve ODE problem

    #Generate Noisy Data
    batchout = batchsol(tvals)        #interpolate ODE solution at specific time points
    NA = batchout[1,:]                #NA at interpolation points, moles
    Random.seed!(rseed[1])
    NA[2:end]  = NA[2:end].*rand(Normal(1.0, noise), length(NA[2:end]))
    NB = batchout[2,:]                #NB at interpolation points, moles
    Random.seed!(rseed[2])
    NB[2:end]  = NB[2:end].*rand(Normal(1.0, noise), length(NB[2:end]))    
    NS = batchout[3,:]                #NS at interpolation points
    yA = NA./10                       #mole fraction of A at interpolation points
    yB = NB./10                       #mole fraction of B at interpolation points
    pA = yA*P                         #partial pressure of A at interpolation points, bar
    pB = yB*P                         #partial pressure of B at interpolation points, bar 
    aA = pA/P0                        #thermodynamic activity of A
    aB = pB/P0                        #thermodynamic activity of B
    CA = pA/R/T                       #CA at interpolation points, mol/L
    CB = pB/R/T                       #CB at interpolation points, mol/L
    XA = (NA0 .- NA)/NA0              #Conversion at interpolation points
    r  = k2(T)*aA                     #reaction rate at interpolation points, 1/s
    return tvals, NA, NB, NS, CA, CB, aA, aB, XA, r, tsol, NAsol, NBsol
end

mainbatch3 (generic function with 1 method)

In [14]:
#Solve the problem, store the returns.
A2     = 2.5e14/1000 #pre-exponential for reaction, 1/s
EA2    = 88.4*1000   #Barrier, J/mol
Ad     = 1e13/1000*1.90 #pre-exponential for deactivation, 1/s
EAd    = 115000      #Barrier for deactivation, J/mol
tset   = [0, 50, 100, 200, 300, 400, 500, 700, 900, 1200, 1500, 1800, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000, 10000] #s
k2(T)  = A2*exp(-EA2/R1/T) #rate constant for reaction, 1/s
kd(T)  = Ad*exp(-EAd/R1/T) #rate constant for deactivation, 1/s

tset     = [0, 150, 300, 600, 900, 1200, 1500, 1800, 2100, 2500, 3000, 3500, 4000, 4500, 5300, 6000, 7000, 8000, 9000, 10000] #s
tset     = tset/10
sites    = [NS0]
rseeds   = [20 21; 22 23; 24 25; 26 27]

sims  = 10

t4  = zeros(length(tset), sims)
t4a = zeros(length(tset), sims)
NA4 = zeros(length(tset), sims)
NB4 = zeros(length(tset), sims)
NS4 = zeros(length(tset), sims)
CA4 = zeros(length(tset), sims)
CB4 = zeros(length(tset), sims)
aA4 = zeros(length(tset), sims)
aB4 = zeros(length(tset), sims)
XA4 = zeros(length(tset), sims)
r4  = zeros(length(tset), sims)

for i = 1:1:sims
    par0   = ComponentArray(
        T = T0, 
        P = P, 
        P0 = P0, 
        R  = R, 
        NA0 = NA0, 
        NB0 = NB0, 
        NS0 = NS0, 
        NS0_l = sites[i], 
        noise = 0, 
        tvals = tset, 
        rseed = [1, 2]
        )
    tvals, NA, NB, NS, CA, CB, aA, aB, XA, rate, tsol, NAsol, NBsol = mainbatch3(par0) #run batch simulation, workup data
    t4[:, i]  = tvals
    t4a[:, i] = tvals .+ tset[end]*(i - 1)
    NA4[:, i] = NA
    NB4[:, i] = NB
    NS4[:, i] = NS
    CA4[:, i] = CA
    CB4[:, i] = CB
    aA4[:, i] = aA
    aB4[:, i] = aB
    XA4[:, i] = XA
    r4[:, i]  = rate
    tvals  = tvals .+ tset[end]*(i - 1)
    sites  = vcat(sites, NS[end])
end

BR02E = Dict("t" => t4, "ta" => t4a, "NA" => NA4, "CA" => CA4, "aA" => aA4, "NS" => NS4)
save_object("BR02E", BR02E)